## Purpose Of this notebook :
  - Build foundational step to get started with LangChain and Database implementations
  - Learn how to leverage the power of Langchain to make database interactions more user-friendly and efficient.

## Install dependencies and libraries

In [ ]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"
! pip install langchain torch transformers sentence_transformers faiss-gpu chainlit ctransformers chromadb bitsandbytes accelerate einops huggingface_hub
! CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose

In [3]:
print("all the  libraries and dependencies successfully installed")

all the  libraries and dependencies successfully installed


## Loading the model

### Load with huggigface hub, and save it to local
- This can be done once, once you have saved the model to your local.
- You can again load that model with the next cell code.
- Model used here is [Llama-7B-chat-GGML](https://huggingface.co/TheBloke/Llama-2-7B-GGML)
- Here is 13b version [Llama-2-13B-chat-GGML](https://huggingface.co/TheBloke/Llama-2-13B-GGML)

In [2]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

model_repo = "TheBloke/Llama-2-13B-chat-GGML"
model_bin_file = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format
local_path_to_save_the_model = "model/"

# In case we need to download model from HF.
model_path_temp = hf_hub_download(repo_id=model_repo, filename=model_bin_file)
print("model download complete")

model download complete


### Save this downloaded model into your local path

In [ ]:
# save this model to path
command_to_transfer_model = f"""{model_path_temp} '{local_path_to_save_the_model}'"""
! cp $command_to_transfer_model

### Load with bin file, from local
- **LlamaCpp** llama-cpp-python is a Python binding for llama.cpp.\
  It supports inference for many LLMs, which can be accessed on HuggingFace.
- We are directly loading the model into langchain llms
- Since here we are using a LLama Model here, we can use LlamaCpp to load directly as langchain llm.
- You can look more about LlamaCpp [Here](https://python.langchain.com/docs/integrations/providers/llamacpp)

In [4]:
from langchain.llms import LlamaCpp
from langchain.agents import load_tools, initialize_agent
model_bin_path = model_path_temp
llm = LlamaCpp(
    model_path=model_bin_path,
    n_threads=2,  # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32, # Change this value based on your model and your GPU VRAM pool.
    n_ctx=2048
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


## Creating DataBase Agent
- Agents use an LLM to decide what actions to take and the order to take them in, making future decisions by iteratively observing the outcome of prior actions.
- LangChain Provides an Sql Database Agent that connect to your SQL database.\
  And can also query your database.

### Creating a dummy database , if you have already have database, You can go to next part

In [5]:
! pip install Faker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 27.8 MB/s eta 0:00:00


In [6]:
from faker import Faker
import pandas as pd
import random
import sqlite3
from smart_open import open
from tabulate import tabulate
def create_sqlite_db_from_csv(db_name,
                              table_name="employee_data",
                              data_df=None,
                              csv_url=None):
    # Connect to the SQLite database (it will be created if it doesn't exist)
    conn = sqlite3.connect(db_name)

    if data_df is None and csv_url is not None:
        # Load CSV file into a Pandas dataframe directly from the URL
        with open(csv_url, "r") as f:
            data_df = pd.read_csv(f)
    data_df.fillna('', inplace=True)
    # Write the dataframe to the SQLite database
    data_df.to_sql(table_name, conn, if_exists="replace", index=False)

    # Commit and close connection
    conn.commit()
    conn.close()
    print(f"Database {db_name} with table {table_name} created successfully!")


class Fake_Bank_Data():
    def __init__(self):
        self.fake = Faker()
        pass

    # Function to generate random dummy data for Customers
    def generate_customers_data(self, rows):
        cols = ["CustomerID", "FirstName", "LastName", "Email"]
        data = []
        for i in range(1, rows + 1):
            data.append(
                [i,
                 self.fake.first_name(),
                 self.fake.last_name(),
                 self.fake.email()
                ])
        return pd.DataFrame(data,columns=cols)

    # Function to generate random dummy data for Transactions
    def generate_transactions_data(self, rows):
        cols = ["TransactionID", "AccountID", "TransactionType", "Amount", "TransactionDate"]
        data = []
        for i in range(1001, 1001 + rows):
            data.append(
                [i,
                 random.randint(101, 200),
                 random.choice(["Deposit", "Withdrawal"]),
                 round(random.uniform(100, 2000), 2),
                 self.fake.date_between(start_date='-30d', end_date='today')
                ]
            )
        return pd.DataFrame(data,columns=cols)

    # Function to generate random dummy data for Accounts
    def generate_accounts_data(self, rows):
        cols = ["AccountID", "CustomerID", "AccountType", "Balance"]
        data = []
        for i in range(101, 101 + rows):
            data.append(
                [i,
                 random.randint(1, 100),
                 random.choice(["Checking", "Savings"]),
                 round(random.uniform(1000, 20000), 2)
                ]
            )
        return pd.DataFrame(data,columns=cols)

    def generate_data(self,
                      customers=20,
                      accounts=20,
                      transactions=20):
        return {'customers': self.generate_customers_data(customers),
                'accounts': self.generate_accounts_data(accounts),
                'transactions':self.generate_transactions_data(transactions)}


bank_fake = Fake_Bank_Data()
df_dict = bank_fake.generate_data(customers=50,
                                  accounts=100,
                                  transactions=200)

# Create SQLite databases from CSV datasets Or DataFrame
for table_name, df in df_dict.items():
    create_sqlite_db_from_csv(data_df=df,
                              db_name="bank-sqlite.db",
                              table_name=table_name)

Database bank-sqlite.db with table customers created successfully!
Database bank-sqlite.db with table accounts created successfully!
Database bank-sqlite.db with table transactions created successfully!


In [14]:
df_dict['customers'].head(2)
print("--"*10)
df_dict['accounts'].head(2)
print("--"*10)
df_dict['transactions'].head(2)

(   CustomerID FirstName LastName                   Email
 0           1    Daniel   Thomas  wandacurry@example.org
 1           2    Connie      Gay     npineda@example.com,
    AccountID  CustomerID AccountType   Balance
 0        101          14    Checking   9813.39
 1        102          86    Checking  15091.31,
    TransactionID  AccountID TransactionType   Amount TransactionDate
 0           1001        194         Deposit  1899.04      2023-10-05
 1           1002        180         Deposit  1711.80      2023-09-18)

In [18]:
df_dict['customers'][df_dict['accounts'].CustomerID.isin(df_dict['customers'].CustomerID)]

<ipython-input-18-51ac56700514>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_dict['customers'][df_dict['accounts'].CustomerID.isin(df_dict['customers'].CustomerID)]


,CustomerID,FirstName,LastName,Email
0,1,Daniel,Thomas,wandacurry@example.org
3,4,Kyle,Davis,whuang@example.org
5,6,James,Crane,bhawkins@example.net
6,7,Danielle,Luna,rdaniels@example.org
10,11,Lindsey,Hahn,kristinasingleton@example.org
11,12,Teresa,Garcia,jasmineward@example.com
12,13,Rachel,Good,melissapotter@example.net
16,17,John,Lewis,debra78@example.com
18,19,Craig,Arias,mgraves@example.org
22,23,Leah,Reynolds,guydiaz@example.org


### Creating Agent

In [24]:
from langchain.agents import AgentExecutor, create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain.sql_database import SQLDatabase

def create_agent(llm,
                 db_uri=None,
                 db=None,
                 toolkit=None,
                 verbose=True,
                 temperature=0.5,
                 handle_parsing_errors=True,
                 **kwargs):
    if db is None:
        db = SQLDatabase.from_uri(db_uri)
    if toolkit is None:
        toolkit = SQLDatabaseToolkit(db=db, llm=llm, sample_rows_in_table_info=1)
    agent = create_sql_agent(llm=llm, toolkit=toolkit, verbose=verbose,
                              handle_parsing_errors=handle_parsing_errors,
                             **kwargs)
    return agent

In [25]:
db_uri = "sqlite:////content/bank-sqlite.db"
handle_parsing_errors = "Always return final answer as string"
# toolkit = SQLDatabaseToolkit(db=db, llm=llm)

default_sql_agent = create_agent(db_uri=db_uri, llm=llm,
                                toolkit=None,
                                verbose=True,
                                handle_parsing_errors=handle_parsing_errors
                                )
print(default_sql_agent.agent.llm_chain.prompt.template)

You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct sqlite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 10 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't k

#### Working - Examples

In [13]:
from langchain.prompts.chat import ChatPromptTemplate
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         """
          You are a helpful AI assistant expert in converting english sentenceto SQL query to find answers to user's question.
          Strictly Follow the below mentioned guidelines while generating response:
          Guideline : Output query MUST be syntactically correct on a sqlite3 database.
          Guideline : Strictly Check syntax of query before returning.
          Guideline : Striclty re-confirm the data types and columns names are correct before returning.
         """
         ),
        ("user", "{question}\n ai: "),
    ]
)

##### Example - It involves querying single table

In [16]:
quest = "what is the email of user Daniel ? "
final_ques = final_prompt.format(
        question=quest)
print(final_ques)
default_sql_agent.run(final_ques)

System: 
          You are a helpful AI assistant expert in converting english sentenceto SQL query to find answers to user's question.
          Strictly Follow the below mentioned guidelines while generating response:
          Guideline : Output query MUST be syntactically correct on a sqlite3 database.
          Guideline : Strictly Check syntax of query before returning.
          Guideline : Striclty re-confirm the data types and columns names are correct before returning.
         
Human: what is the email of user Daniel ? 
 ai: 


> Entering new AgentExecutor chain...


Llama.generate: prefix-match hit


Action: sql_db_schema
Action Input: table1, table2, table3
Observation: Error: table_names {'table3', 'table2', 'table1'} not found in database
Thought:

Llama.generate: prefix-match hit


 Oops! It looks like none of those tables exist in the database.  I should check the list of available tables.
Action: sql_db_list_tables
Action Input: empty string
Observation: accounts, customers, transactions
Thought:

...

Llama.generate: prefix-match hit


 Ah ha! Now I know the correct query.  Let me try it again.
Action: sql_db_query
Action Input: SELECT email FROM customers WHERE CustomerID = 1
Observation: [('wandacurry@example.org',)]
Thought:

Llama.generate: prefix-match hit


 Yes! I was able to find the email of 'Daniel'. The final answer is wandacurry@example.org.
Final Answer: wandacurry@example.org

End!

> Finished chain.


'wandacurry@example.org\n\nEnd!'

##### Example - It involves multiple tables, difficulty level medium

In [19]:
quest = "what is the account balance of user Lindsey ? "
final_ques = final_prompt.format(
        question=quest)
print(final_ques)
default_sql_agent.run(final_ques)

System: 
          You are a helpful AI assistant expert in converting english sentenceto SQL query to find answers to user's question.
          Strictly Follow the below mentioned guidelines while generating response:
          Guideline : Output query MUST be syntactically correct on a sqlite3 database.
          Guideline : Strictly Check syntax of query before returning.
          Guideline : Striclty re-confirm the data types and columns names are correct before returning.
         
Human: what is the account balance of user Lindsey ? 
 ai: 


> Entering new AgentExecutor chain...


Llama.generate: prefix-match hit


Action: sql_db_schema
Action Input: table1, table2
Observation: Error: table_names {'table2', 'table1'} not found in database
Thought:

Llama.generate: prefix-match hit


 Hmm... It seems that the table names given are not in the database. Let me look at the list of tables to see which ones I have available.
Action: sql_db_list_tables
Action Input: 
Observation: accounts, customers, transactions
Thought:

...

Llama.generate: prefix-match hit


 Now that I have the schema of each table, I can determine which tables are most relevant to the user's question and what columns I need to query. Let me check the account balance for Lindsey.
Action: sql_db_query
Action Input: select Balance from accounts where CustomerID = '14'
Observation: [(9813.39,)]
Thought:

Llama.generate: prefix-match hit


 Great! I found the account balance of Lindsey.
Final Answer: The account balance of Lindsey is 9813.39.

> Finished chain.


'The account balance of Lindsey is 9813.39.'

##### Example - It involves multiple tables, difficulty level hard
-  It can be countered by giving more clear prompts
- You can customize the agent also, for more custom use-case

In [15]:
quest = "which user have maximum account balance ? , what is his firstname ? "
final_ques = final_prompt.format(
        question=quest)
print(final_ques)
default_sql_agent.run(final_ques)

System: 
          You are a helpful AI assistant expert in converting english sentenceto SQL query to find answers to user's question.
          Strictly Follow the below mentioned guidelines while generating response:
          Guideline : Output query MUST be syntactically correct on a sqlite3 database.
          Guideline : Strictly Check syntax of query before returning.
          Guideline : Striclty re-confirm the data types and columns names are correct before returning.
         
Human:  which user have maximum account balance ? , what is his firstname ? 
 ai: 


> Entering new AgentExecutor chain...
Action:  sql_db_schema  'mytable'
Action Input:  'mytable'
Observation: sql_db_schema  'mytable' is not a valid tool, try one of [sql_db_query, sql_db_schema, sql_db_list_tables, sql_db_query_checker].
Thought:

Llama.generate: prefix-match hit


 Hmm, I wonder if the table name is correct. Let me double check.
Action:  sql_db_list_tables
Action Input:  ''
Observation: accounts, customers, transactions
Thought:

Llama.generate: prefix-match hit


 Ah ha! There's the table 'accounts'. Now I need to query the schema of that table.
Action:  sql_db_schema 'accounts'
Action Input:  'accounts'
Observation: sql_db_schema 'accounts' is not a valid tool, try one of [sql_db_query, sql_db_schema, sql_db_list_tables, sql_db_query_checker].
Thought:

...

Llama.generate: prefix-match hit


 Hmm, I wonder if the query is correct now. Let me check.
Action:  sql_db_query 'SELECT firstname FROM accounts WHERE balance = (SELECT MAX(balance) FROM accounts GROUP BY balance ORDER BY balance DESC LIMIT 1);'
Action Input:  'SELECT firstname FROM accounts WHERE balance = (SELECT MAX(balance) FROM accounts GROUP BY balance ORDER BY balance DESC LIMIT 1);'
Observation: sql_db_query 'SELECT firstname FROM accounts WHERE balance = (SELECT MAX(balance) FROM accounts GROUP BY balance ORDER BY balance DESC LIMIT 1);' is not a valid tool, try one of [sql_db_query, sql_db_schema, sql_db_list_tables, sql_db_query_checker].
Thought:

Llama.generate: prefix-match hit


 Oh dear, it looks like the query is still incorrect. I think I need to re-evaluate my approach.
Final Answer:  I don't know.

Please let me know if there is any other question you have or if there is anything else i can do to assist you.

> Finished chain.


"I don't know.\n\nPlease let me know if there is any other question you have or if there is anything else i can do to assist you."

#### Creating Custom Agent For Advance / Custom use

 - #### Docs Refferences -
    - LangChain Home [Source Code Docs](https://api.python.langchain.com/en/latest/api_reference.html#module-langchain.agents)
    - Create SQL Agent Methos [Source Code Docs Here](https://api.python.langchain.com/en/latest/_modules/langchain/agents/agent_toolkits/sql/base.html#create_sql_agent)
    - ZeroShotAgent [Source Code Docs Here](https://api.python.langchain.com/en/latest/_modules/langchain/agents/mrkl/base.html#ZeroShotAgent)
    - Tools in Sqltoolkit [Source Code Docs Here](https://api.python.langchain.com/en/latest/_modules/langchain/tools/sql_database/tool.html#QuerySQLCheckerTool)
    - Langchain Home [Github Here](https://github.com/langchain-ai/langchain/tree/master/libs/langchain/langchain)
    - Prompts used in Create Sql Agent [Github here](https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/agents/agent_toolkits/sql/prompt.py)
    - Output Parser Used [Github Here](https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/agents/mrkl/output_parser.py)

##### Creating Custom Classes For Tools
- In this I am customizing the agent to just return the query,
- for now I am making changes in the default classes and methods.
- Anyone can customize it further, or can create a complete new agant out of this.

In [29]:
"""Toolkit for interacting with an SQL database."""
from typing import List

from langchain.agents.agent_toolkits.base import BaseToolkit
from langchain.pydantic_v1 import Field
from langchain.schema.language_model import BaseLanguageModel
from langchain.tools import BaseTool
from langchain.tools.sql_database.tool import (
    InfoSQLDatabaseTool,
    ListSQLDatabaseTool,
    QuerySQLCheckerTool,
    QuerySQLDataBaseTool,
)
from langchain.utilities.sql_database import SQLDatabase


from typing import Any, Dict, Optional
from langchain.pydantic_v1 import BaseModel, Extra, Field, root_validator

from langchain.schema.language_model import BaseLanguageModel
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.utilities.sql_database import SQLDatabase
from langchain.tools.base import BaseTool
from langchain.tools.sql_database.prompt import QUERY_CHECKER

class BaseSQLDatabaseTool(BaseModel):
    """Base tool for interacting with a SQL database."""

    db: SQLDatabase = Field(exclude=True)

    # Override BaseTool.Config to appease mypy
    # See https://github.com/pydantic/pydantic/issues/4173
    class Config(BaseTool.Config):
        """Configuration for this pydantic object."""

        arbitrary_types_allowed = True
        extra = Extra.forbid

class MyQuerySQLCheckerTool(BaseSQLDatabaseTool, BaseTool):
    """Use an LLM to check if a query is correct.
    Adapted from https://www.patterns.app/blog/2023/01/18/crunchbot-sql-analyst-gpt/"""

    template: str = QUERY_CHECKER
    llm: BaseLanguageModel
    llm_chain: LLMChain = Field(init=False)
    name: str = "sql_db_query_checker"
    description: str = """
    "Use this tool to double check if your query is correct
    output is the corrected query in string format
    Strictly return the input query , if it is taking too much time to correct it
    Strictly return the input query , You are not able to correct it"
    """

    @root_validator(pre=True)
    def initialize_llm_chain(cls, values: Dict[str, Any]) -> Dict[str, Any]:
        if "llm_chain" not in values:
            values["llm_chain"] = LLMChain(
                llm=values.get("llm"),
                prompt=PromptTemplate(
                    template=QUERY_CHECKER, input_variables=["query", "dialect"]
                ),
            )

        if values["llm_chain"].prompt.input_variables != ["query", "dialect"]:
            raise ValueError(
                "LLM chain for QueryCheckerTool must have input variables ['query', 'dialect']"
            )

        return values

    def _run(
        self,
        query: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the LLM to check the query."""
        return self.llm_chain.predict(
            query=query,
            dialect=self.db.dialect,
            callbacks=run_manager.get_child() if run_manager else None,
        )

    async def _arun(
        self,
        query: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        return await self.llm_chain.apredict(
            query=query,
            dialect=self.db.dialect,
            callbacks=run_manager.get_child() if run_manager else None,
        )


class MyQuerySQLDataBaseTool(BaseSQLDatabaseTool, BaseTool):
    """Tool for querying a SQL database."""

    name: str = "sql_db_query"
    description: str = """
    "Input to this tool is a detailed and correct SQL query,
    output is a Strictly do not execute the query.
    Just return the input query in highlighted string format "
    """

    def _run(
        self,
        query: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Execute the query, return the results or an error message."""
        # return self.db.run_no_throw(query)
        return query


class MySQLDatabaseToolkit(BaseToolkit):
    """Toolkit for interacting with SQL databases."""

    db: SQLDatabase = Field(exclude=True)
    llm: BaseLanguageModel = Field(exclude=True)

    @property
    def dialect(self) -> str:
        """Return string representation of SQL dialect to use."""
        return self.db.dialect

    class Config:
        """Configuration for this pydantic object."""

        arbitrary_types_allowed = True

    def get_tools(self) -> List[BaseTool]:
        """Get the tools in the toolkit."""
        list_sql_database_tool = ListSQLDatabaseTool(db=self.db)
        info_sql_database_tool_description = (
            "Input to this tool is a comma-separated list of tables, output is the "
            "schema and sample rows for those tables. "
            "Be sure that the tables actually exist by calling "
            f"{list_sql_database_tool.name} first! "
            "Example Input: 'table1, table2, table3'"
        )
        info_sql_database_tool = InfoSQLDatabaseTool(
            db=self.db, description=info_sql_database_tool_description
        )
        # query_sql_database_tool_description = (
        #     "Input to this tool is a detailed and correct SQL query, output is a "
        #     "result from the database. If the query is not correct, an error message "
        #     "will be returned. If an error is returned, rewrite the query, check the "
        #     "query, and try again. If you encounter an issue with Unknown column "
        #     f"'xxxx' in 'field list', using {info_sql_database_tool.name} "
        #     "to query the correct table fields."
        # )

        query_sql_database_tool_description = (
            "Input to this tool is a detailed and correct SQL query, output is a "
            "Strictly do not execute the query."
            "Just return the input query in highlighted string format "
        )
        query_sql_database_tool = MyQuerySQLDataBaseTool(
            db=self.db, description=query_sql_database_tool_description
        )
        # query_sql_checker_tool_description = (
        #     "Use this tool to double check if your query is correct before executing "
        #     "it. Always use this tool before executing a query with "
        #     f"{query_sql_database_tool.name}!"
        # )
        query_sql_checker_tool_description = (
            "Use this tool to double check if your query is correct "
            "output is the corrected query in string format"
            "Strictly return the input query , if it is taking too much time to correct it"
            "Strictly return the input query , You are not able to correct it"

            f"{query_sql_database_tool.name}!"
        )
        query_sql_checker_tool = QuerySQLCheckerTool(
            db=self.db, llm=self.llm, description=query_sql_checker_tool_description
        )
        return [
            # query_sql_database_tool,
            info_sql_database_tool,
            list_sql_database_tool,
            query_sql_checker_tool,
        ]

##### Creating Custom Method For Agent and custom Prompts

In [32]:
"""SQL agent."""
import re
from typing import Any, Dict, List, Optional, Sequence

from langchain.agents.agent import AgentExecutor, BaseSingleActionAgent, AgentOutputParser
from langchain.agents.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain.agents.mrkl.base import ZeroShotAgent
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.callbacks.base import BaseCallbackManager
from langchain.chains.llm import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.schema.language_model import BaseLanguageModel
from langchain.schema.messages import AIMessage, SystemMessage
from langchain.tools import BaseTool

from langchain.schema import (
    AgentAction,
    AgentFinish,
    BaseOutputParser,
    BasePromptTemplate,
    OutputParserException,
)

from typing import (
    Any,
    Callable,
    Dict,
    List,
    Optional,
    Sequence,
    Tuple,
    Union,
)
SQL_PREFIX = """You are an agent designed to convert the statement in english language into a SQL query..
Given an input question, create a syntactically correct sqlite query to run.
You can order the results by a relevant column to return the most interesting examples in the database.
You Do not have access to tools for interacting with the database.
Strictly just prepare the query.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query with schema provided, if any mismatch happend, rewrite the query and try again.
"""

SQL_SUFFIX = """Begin!

Question: {input}
Thought: I should think and guess if the query will work and matching with what human asked, if yes then return the query
{agent_scratchpad}"""

SQL_FUNCTIONS_SUFFIX = """I should look for the table names, columns names to build the query.
I should look for constaraints provided for columns to build the query.
I should look for column names that to be included in final query"""

FORMAT_INSTRUCTIONS = """
Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat 6 times)
... Strictly You should stop the, once guessing the query.
Thought: I now know the final answer.
Final Answer: the final answer to the original input question always return and highlight the guessed query in string format."""

class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output or 'I should check the query syntax with another tool' in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise OutputParserException(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

def create_sql_agent(
    llm: BaseLanguageModel,
    toolkit: SQLDatabaseToolkit,
    agent_type: AgentType = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    callback_manager: Optional[BaseCallbackManager] = None,
    prefix: str = SQL_PREFIX,
    suffix: Optional[str] = None,
    format_instructions: str = FORMAT_INSTRUCTIONS,
    input_variables: Optional[List[str]] = None,
    top_k: int = 10,
    max_iterations: Optional[int] = 15,
    max_execution_time: Optional[float] = None,
    early_stopping_method: str = "force",
    verbose: bool = False,
    agent_executor_kwargs: Optional[Dict[str, Any]] = None,
    extra_tools: Sequence[BaseTool] = (),
    **kwargs: Dict[str, Any],
) -> AgentExecutor:
    """Construct an SQL agent from an LLM and tools."""
    tools = toolkit.get_tools() + list(extra_tools)
    prefix = prefix.format(dialect=toolkit.dialect, top_k=top_k)
    agent: BaseSingleActionAgent

    if agent_type == AgentType.ZERO_SHOT_REACT_DESCRIPTION:
        prompt = ZeroShotAgent.create_prompt(
            tools,
            prefix=prefix,
            suffix=suffix or SQL_SUFFIX,
            format_instructions=format_instructions,
            input_variables=input_variables,
        )
        llm_chain = LLMChain(
            llm=llm,
            prompt=prompt,
            callback_manager=callback_manager,
        )
        tool_names = [tool.name for tool in tools]
        agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names, **kwargs)

    elif agent_type == AgentType.OPENAI_FUNCTIONS:
        messages = [
            SystemMessage(content=prefix),
            HumanMessagePromptTemplate.from_template("{input}"),
            AIMessage(content=suffix or SQL_FUNCTIONS_SUFFIX),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
        input_variables = ["input", "agent_scratchpad"]
        _prompt = ChatPromptTemplate(input_variables=input_variables, messages=messages)

        agent = OpenAIFunctionsAgent(
            llm=llm,
            prompt=_prompt,
            tools=tools,
            callback_manager=callback_manager,
            **kwargs,
        )
    else:
        raise ValueError(f"Agent type {agent_type} not supported at the moment.")

    return AgentExecutor.from_agent_and_tools(
        agent=agent,
        tools=tools,
        callback_manager=callback_manager,
        verbose=verbose,
        max_iterations=max_iterations,
        max_execution_time=max_execution_time,
        early_stopping_method=early_stopping_method,
        **(agent_executor_kwargs or {}),
    )

def create_agent(llm,
                 db_uri=None,
                 db=None,
                 toolkit=None,
                 verbose=True,
                 temperature=0.5,
                 handle_parsing_errors=True,
                 **kwargs):
    if db is None:
        db = SQLDatabase.from_uri(db_uri)
    if toolkit is None:
        toolkit = SQLDatabaseToolkit(db=db, llm=llm, sample_rows_in_table_info=1)
    agent = create_sql_agent(llm=llm, toolkit=toolkit, verbose=verbose,
                              handle_parsing_errors=handle_parsing_errors,
                             **kwargs)
    return agent

#### Creating custom agent

In [33]:
db_uri = "sqlite:////content/bank-sqlite.db"
handle_parsing_errors = "Always return final answer as string"
db = SQLDatabase.from_uri(db_uri)

my_toolkit = MySQLDatabaseToolkit(db=db, llm=llm)
custom_sql_agent = create_agent(db_uri=db_uri,
                                db=db,
                                llm=llm,
                                toolkit=my_toolkit,
                                verbose=True,
                                handle_parsing_errors=handle_parsing_errors
                                )
print(custom_sql_agent.agent.llm_chain.prompt.template)

You are an agent designed to convert the statement in english language into a SQL query..
Given an input question, create a syntactically correct sqlite query to run.
You can order the results by a relevant column to return the most interesting examples in the database.
You Do not have access to tools for interacting with the database.
Strictly just prepare the query.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query with schema provided, if any mismatch happend, rewrite the query and try again.


sql_db_schema: Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: 'table1, table2, table3'
sql_db_list_tables: Input is an empty string, output is a comma separated list of tables in the database.
sql_db_query_checker: Use this tool to double check if 

#### Working - Examples

##### Example , involves single column

In [36]:
quest = "write me a query to get - what is the email of customer, who's first name is Daniel ? "
final_ques = final_prompt.format(
        question=quest)
print(final_ques)
custom_sql_agent.run(final_ques)

System: 
          You are a helpful AI assistant expert in converting english sentenceto SQL query to find answers to user's question.
          Strictly Follow the below mentioned guidelines while generating response:
          Guideline : Output query MUST be syntactically correct on a sqlite3 database.
          Guideline : Strictly Check syntax of query before returning.
          Guideline : Striclty re-confirm the data types and columns names are correct before returning.
         
Human: write me a query to get - what is the email of customer, who's first name is Daniel ? 
 ai: 


> Entering new AgentExecutor chain...


Llama.generate: prefix-match hit


Action: sql_db_query_checker
Action Input: select email from customers where first_name='Daniel';

Llama.generate: prefix-match hit



Observation: 
--------------

```sql
SELECT email FROM customers WHERE first_name = 'Daniel';
```
Thought:

...

Llama.generate: prefix-match hit



Observation: 

select email from customers where first_name='Daniel';
Thought:

Llama.generate: prefix-match hit


 My guess is correct!
Final Answer: select email from customers where first_name='Daniel';

> Finished chain.


"select email from customers where first_name='Daniel';"

##### Example - Involves multiple table, difficulty level medium

In [38]:
quest = "write me a query to get - the balance of customer from accounts who's first name is Daniel. \
you can use customerId column as joining condition"
final_ques = final_prompt.format(
        question=quest)
print(final_ques)
custom_sql_agent.run(final_ques)

System: 
          You are a helpful AI assistant expert in converting english sentenceto SQL query to find answers to user's question.
          Strictly Follow the below mentioned guidelines while generating response:
          Guideline : Output query MUST be syntactically correct on a sqlite3 database.
          Guideline : Strictly Check syntax of query before returning.
          Guideline : Striclty re-confirm the data types and columns names are correct before returning.
         
Human: write me a query to get - the balance of customer from accounts who's first name is Daniel. you can use customerId column as joining condition
 ai: 


> Entering new AgentExecutor chain...


Llama.generate: prefix-match hit


Action: sql_db_query_checker
Action Input: the input query that i am going to check "SELECT balance FROM customer WHERE firstname = 'Daniel'"

Llama.generate: prefix-match hit



Observation: 
SELECT balance FROM customer WHERE firstname = 'Daniel'

Input Query: 
SELECT balance FROM customer WHERE firstname = 'Daniel'
Thought:

...

Llama.generate: prefix-match hit


 Great! The query is correct, I can now run it on the database.
Action: sql_db_list_tables
Action Input:
Observation: accounts, customers, transactions
Thought:

Llama.generate: prefix-match hit


 Now I can join the tables and get the balance for customer with firstname 'Daniel'
Action: 
Final Answer: "SELECT balance FROM customer WHERE firstname = 'Daniel'"
the above query will return the balance of the customer with firstname as Daniel.

> Finished chain.


'"SELECT balance FROM customer WHERE firstname = \'Daniel\'"\nthe above query will return the balance of the customer with firstname as Daniel.'

# NOTE -
This custom agent is not perfect, It needs improvemnts.